In [1]:
# Imports.
import sys
print(sys.version)
print()
from collections import deque
import matplotlib.pyplot as plt
import gym
import random
import time
import numpy as np
from tqdm.notebook import trange
import tensorflow as tf
print("Tensorflow", tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
print(tf.config.list_physical_devices('GPU'))
print()
import slimevolleygym

3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]

Tensorflow 2.5.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



In [16]:
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_network
from tf_agents.networks import value_rnn_network
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.agents import PPOClipAgent
from tf_agents.environments import suite_gym
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.environments import utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

In [3]:
from IPython import display
from pyvirtualdisplay import Display
displayy = Display(visible=0, size=(1, 1))
displayy.start()
%matplotlib inline

# Parameters

In [4]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

# Create Environment

In [5]:
# Create TF Environment around SlimeVolleyball.
class TFSlimeVolley(py_environment.PyEnvironment):
    def __init__(self):
        #self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int8, name='action', minimum=0, maximum=1)
        #self._observation_spec = ts.TimeStep({'discount': array_spec.BoundedArraySpec(
        #    shape=(), dtype=np.float32, name='discount', minimum=0, maximum=1),
        #                                     'observation': array_spec.BoundedArraySpec(
        #    shape=(12,), dtype=np.float32, name='observation', minimum=np.finfo(np.float32).min, maximum=np.finfo(np.float32).max),
        #                         'reward': array_spec.ArraySpec(shape=(), dtype=np.float32, name='reward'),
        #                         'step_type': array_spec.ArraySpec(shape=(), dtype=np.int32, name='step_type')})
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(12,), dtype=np.float32, minimum=np.finfo(np.float32).min,
            maximum=np.finfo(np.float32).max, name='observation')
        
        self.env = gym.make('SlimeVolley-v0')
        self._episode_ended = False
    def action_spec(self):
        return self._action_spec
    def observation_spec(self):
        return self._observation_spec
    
    def _reset(self):
        self._episode_ended = False
        return ts.restart(np.array(self.env.reset(), dtype=np.float32))
    def _step(self, action):
        if self._episode_ended:
            return self.reset()
        action_reshape = np.zeros(3)
        action_reshape[action] = 1
        observation, reward, episode_done, info = self.env.step(action_reshape)
        if episode_done:
            self._episode_ended = True
            return ts.termination(np.array(observation, dtype=np.float32), reward)
        else:
            return ts.transition(np.array(observation, dtype=np.float32), reward=reward, discount=1.0)
        
    


In [6]:
#env_name = 'CartPole-v0'
#env = suite_gym.load(env_name)
#py_env = tf_py_environment.TFPyEnvironment(env)
#py_env.action_spec()
#time_spec = py_env.time_step_spec()
#py_env.reset()

#env = gym.make('SlimeVolley-v0')
#print("Observation Space " + str(env.observation_space.shape))
#print("Action Space " + str(env.action_space.shape))

In [7]:
#env.action_space.sample()
#env.observation_space.sample()
#env.reset()
#env.step([1,0,0])
#env.current_time_step()

In [8]:
train_env = tf_py_environment.TFPyEnvironment(TFSlimeVolley())
eval_env = tf_py_environment.TFPyEnvironment(TFSlimeVolley())
#print(train_env.time_step_spec())
#print(train_env.reset())

In [10]:
utils.validate_py_environment(TFSlimeVolley(), episodes=5)

In [19]:
train_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(2, dtype=int32))


# Create Actor and Value net.

In [11]:
# Create Actor and Value net.
fc_layer_params = (200, 100)
actor_fc_layers = (200, 100)
value_fc_layers = (200, 100)
lstm_size=(20,)

# actor_net = actor_distribution_rnn_network.ActorDistributionRnnNetwork(
#   tf_env.observation_spec(),
#   tf_env.action_spec(),
#   input_fc_layer_params=actor_fc_layers,
#   output_fc_layer_params=None,
#   lstm_size=lstm_size)
# value_net = value_rnn_network.ValueRnnNetwork(
#   tf_env.observation_spec(),
#   input_fc_layer_params=value_fc_layers,
#   output_fc_layer_params=None)
actor_net = actor_distribution_network.ActorDistributionNetwork(
  train_env.observation_spec(),
  train_env.action_spec(),
  fc_layer_params=actor_fc_layers,
  activation_fn=tf.keras.activations.tanh)
value_net = value_network.ValueNetwork(
  train_env.observation_spec(),
  fc_layer_params=value_fc_layers,
  activation_fn=tf.keras.activations.tanh)

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = PPOClipAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    actor_net=actor_net,
    value_net=value_net,
    optimizer=optimizer,
    train_step_counter=train_step_counter)

agent.initialize()

eval_policy = agent.policy
collect_policy = agent.collect_policy

# Random policy to collect initial data.
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

In [13]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=1,
    max_length=replay_buffer_max_length)

# Metrics and Evaluation

In [14]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

class BestModel:
    def __init__(self, agent):
        self.best_model = keras.models.clone_model(agent.model)
        self.best_model.set_weights(agent.model.get_weights())
        self.state_num = agent.state_num
        self.best_score = -10
        self.update_index = 0
    def UpdateBest(self, agent, eval_score, i):
        if (eval_score > self.best_score):
            self.best_model.set_weights(agent.model.get_weights())
            self.best_score = eval_score
            self.update_index = i
            print("New best!")
    def RunBest(self, state):
        state = np.reshape(state, [1, self.state_num])
        action = self.best_model(state, training=False)
        return np.argmax(action[0])

#bmodel = BestModel(agent)

## Run Training

In [15]:
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

# This loop is so common in RL, that we provide standard implementations. 
# For more details see tutorial 4 or the drivers module.
# https://github.com/tensorflow/agents/blob/master/docs/tutorials/4_drivers_tutorial.ipynb 
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset
iterator = iter(dataset)
print(iterator)
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

TypeError: The two structures do not match:
  Trajectory(
{'action': .,
 'discount': .,
 'next_step_type': .,
 'observation': .,
 'policy_info': (),
 'reward': .,
 'step_type': .})
vs.
  Trajectory(
{'action': .,
 'discount': .,
 'next_step_type': .,
 'observation': .,
 'policy_info': DictWrapper({'dist_params': DictWrapper({'logits': .})}),
 'reward': .,
 'step_type': .})
Values:
  Trajectory(
{'action': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'next_step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>,
 'observation': <tf.Tensor: shape=(1, 12), dtype=float32, numpy=array([[ 1.2  ,  0.15 ,  0.   ,  0.   ,  0.   ,  1.2  , -1.182,  1.94 ,  1.2  ,  0.15 ,  0.   ,  0.   ]], dtype=float32)>,
 'policy_info': (),
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, nump

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in trange(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

In [ ]:
iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=5.5, bottom=-5.5)

## Watching a Game

In [ ]:
obs = env.reset()
done = False
total_reward = 0
img = plt.imshow(env.render(mode='rgb_array'))

while not done:
    action = bmodel.RunBest(obs)
    action_reshaped = np.zeros(agent.action_num)
    action_reshaped[action] = 1
    obs, reward, done, info = env.step(action_reshaped)
    total_reward += reward
    img.set_data(env.render(mode='rgb_array'))
    display.clear_output(wait=True)
    display.display(plt.gcf())

print("score:", total_reward)
print(bmodel.update_index)